In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

In [10]:
import logging
from pathlib import Path
from src.data.WordData import WordData
import json
import numpy as np
from src.models.WordDataLoader import WordDataLoader
from src.models.simpleLSTMModel import simpleLSTMModel
from src.models.WordDataLoader import IdWordDataLoader
from src.models.ModelTrainer import ModelTrainer
from src.models.TweetGenerator import TweetGenerator
from src.models.GenerateTweetCallback import GenerateTweetCallback
from keras.models import load_model
import pandas as pd

In [11]:
with open('../data/interim/tokens_2.json') as f:
    tokens = json.load(f)

tweets_tokens = pd.read_csv('../data/interim/tweets_2.csv')
tokens_ss_pbm = tweets_tokens[(tweets_tokens.count_hashtag_ano <= 2)  & (tweets_tokens.count_mention_ano <= 1)].index


with open('../data/interim/tokens_id_keep_2.json') as f:
    tokens_id_keep = json.load(f)

tokens_id_keep = list(set(tokens_id_keep) & set(tokens_ss_pbm))
print(len(tokens_id_keep))
                            
word_data = WordData(tokens, tokens_id_keep, 0)

# print(word_data.token_final[1:4])

print(word_data.getVocabularyLength())

# print(word_data.ref_word_to_id)

sentence_loader = IdWordDataLoader(20, 5, word_data, 1, 1)

# for x,y in word_data_loader.generate():
# 	print(x)
# 	print(y)
# 	break

lstm_model = load_model('../models/simpleLSTMModel/version3/model-160.hdf5')
# lstm_model = simpleLSTMModel(word_data.getVocabularyLength(), LSTM_HIDDEN_SIZE, (None, None))
# print(lstm_model.model.summary())

# lstm_model.save()
# callbacks = [generate_tweet_callback]
# trainer = ModelTrainer(lstm_model, word_data_loader, NB_EPOCH, workers = 3, use_multiprocessing = True, callbacks = callbacks)
# trainer.train()

# lstm_model.save_directory = '../models/simpleLSTMModel/version1'
# lstm_model.load('model-60')
print(lstm_model.model.summary())

23744
26487
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         7946100   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 300)         721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 300)         721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 26487)       7972587   
_________________________________________________________________
activation_1 (Activation)    (None, None, 26487)       0         
Total params: 17,361,087
Trainable params: 17,361,087
Non-trainable params: 0
____________________________________________________

/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [12]:
tweet_generator = TweetGenerator(lstm_model, word_data)
t = tweet_generator.tweet()

[26484]


/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [13]:
' '.join(t)

'#paris2024 : la candidature de paris pour les #jo2024 _SENTENCE_STOP_'

In [14]:
t = tweet_generator.randomTweet(stop = 100)

[3512]


/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [15]:
' '.join(t)

'restons pour #paris2024 _SENTENCE_STOP_'

In [24]:
phrase = ['je']
id_token = [word_data.ref_word_to_id[word_data.sentence_token_start]]
for mot in phrase:
    id_token.append(word_data.ref_word_to_id[mot])

In [25]:
' '.join(tweet_generator.tweet(np.array(id_token)))

[26484    49]


/home/prise6/anaconda3/envs/py35/lib/python3.5/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


"_SENTENCE_START_ je suis contre les #jo2024 a paris , c' est pour les #jo2024 _SENTENCE_STOP_"